<a href="https://colab.research.google.com/github/Migliorin/COVID_Project/blob/main/Projeto_Analise_Similariedade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers
!brew install libomp # if you are on OSX, for faiss
!pip install transformers faiss torch
!sudo apt-get install libgomp1
!pip install faiss-cpu --no-cache
!pip3 install sent2vec

# Importes

In [ ]:
import json
import pandas as pd

# Importes transformers
import numpy as np
from transformers import AutoModel, AutoTokenizer
import torch
import faiss

# BERT
from scipy import spatial
from sent2vec.vectorizer import Vectorizer

# Abertura do dataset

In [ ]:
with open('/content/drive/Shareddrives/COVID - Projeto/dataset_v1.json','r') as f:
    df_json = json.load(f)

In [ ]:
perguntas = []
for i in df_json['data'][:1]:
    for j in i['paragraphs']:
        for k in j['qas']:
            perguntas.append(k['question'])

In [ ]:
print(len(perguntas),perguntas[:5])

767 ['Quando Beyonce começou a se tornar popular?', 'Em quais áreas Beyonce competiu quando ela estava crescendo?', "Quando Beyoncé deixou Destiny's Child e se tornou uma cantora solo?", 'Em que cidade e estado Beyonce cresceu?', 'Em qual década Beyonce se tornou famosa?']


# Utilizando transformers

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
documents = perguntas

vectors = [
  # tokenize the document, return it as PyTorch tensors (vectors),
  # and pass it onto the model
  model(**tokenizer(document, return_tensors='pt'))[0].detach().squeeze()
  for document in documents
]

#[v.size() for v in vectors]

In [ ]:
aux = [v.size() for v in vectors]
aux[:3]

[torch.Size([12, 768]), torch.Size([14, 768]), torch.Size([18, 768])]

In [ ]:
averaged_vectors = [torch.mean(vector, dim=0) for vector in vectors]

In [ ]:
def encode(document: str) -> torch.Tensor:
  tokens = tokenizer(document, return_tensors='pt')
  vector = model(**tokens)[0].detach().squeeze()
  return torch.mean(vector, dim=0)

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768)) # the size of our vector space
# index all the documents, we need them as numpy arrays first
index.add_with_ids(
    np.array([t.numpy() for t in averaged_vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(documents)))
)

def search(query: str, k=1):
  encoded_query = encode(query).unsqueeze(dim=0).numpy()
  top_k = index.search(encoded_query, k)
  scores = top_k[0][0]
  results = [documents[_id] for _id in top_k[1][0]]
  return list(zip(results, scores))

In [ ]:
perguntas_teste[1]

'Com que música Beyonce contribuiu para a campanha?'

In [ ]:
search("Onde beyonce nasceu?",k=3)

[('Onde Blue Ivy nasceu?', 33.20627),
 ('Onde Beyonce se apresentou em 2011?', 32.617737),
 ('Quando Beyonce começou a se tornar popular?', 32.591763)]

# Usando BERT

In [ ]:
sentences = perguntas

vectorizer = Vectorizer()
vectorizer.bert(sentences)

In [ ]:
vectorizer.__dict__

{'vectors': array([[-0.16396935, -0.27253982,  0.14399289, ...,  0.05619444,
          0.4985507 ,  0.18187934],
        [-0.23593178, -0.3083715 ,  0.19492105, ...,  0.10151217,
          0.40506238,  0.20356336],
        [-0.20170058, -0.36552688,  0.08919662, ...,  0.09863595,
          0.44682708,  0.13105403],
        ...,
        [-0.15903749, -0.18028979,  0.13669719, ...,  0.10021325,
          0.49533468,  0.28977045],
        [-0.13491194, -0.28364253,  0.19597052, ...,  0.06666869,
          0.3542019 ,  0.18729764],
        [-0.13491194, -0.28364253,  0.19597052, ...,  0.06666869,
          0.3542019 ,  0.18729764]], dtype=float32)}

In [ ]:
vectors_bert = vectorizer.vectors

In [ ]:
p = procura('Onde beyonce morou?')
print("Distancia: {:.4f}\nPergunta próxima: {}".format(p[0],sentences[p[1]]))

Distancia: 0.2942
Pergunta próxima: O que B.I.C. apoiar?


# Similaridade

## Por distância (TF)

### Importes

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### TF

In [ ]:
# Documentos
C = ["notícias sobre","notícias sobre campanha de comidas orgânicas", "notícias da campanha presidencial",
     "notícias da campanha presidencial, eleição presidencial.", "notícias sobre campanha de comidas orgânicas, campanha, campanha, campanha",
     "notícias sobre campanha presidencial."]

q = C[-1] # Query

stopWords = stopwords.words('portuguese')

In [ ]:
vectorizer = CountVectorizer(stop_words = stopWords)
tf_docs = vectorizer.fit_transform(C).toarray() # Bag of words dos documentos
tf_query = tf_docs[-1][:] # Bag of words da query
tf_docs = tf_docs[:-1][:] # Bag of words dos documentos

In [ ]:
print('TF de cada documento:\n',tf_docs)
print('\nTF da query:\n',tf_query)
print('\n')

TF de cada documento:
 [[0 0 0 1 0 0 1]
 [1 1 0 1 1 0 1]
 [1 0 0 1 0 1 0]
 [1 0 1 1 0 2 0]
 [4 1 0 1 1 0 1]]

TF da query:
 [1 0 0 1 0 1 1]




In [ ]:
for doc, i in zip(C[:-1],range(tf_docs.shape[0])):
    f = np.dot(tf_docs[i][:],tf_query) # Produto interno por meio de matrix entre a query e o documento
    print('f("{}","{}") = {}'.format(q,doc,f))

f("notícias sobre campanha presidencial.","notícias sobre") = 2
f("notícias sobre campanha presidencial.","notícias sobre campanha de comidas orgânicas") = 3
f("notícias sobre campanha presidencial.","notícias da campanha presidencial") = 3
f("notícias sobre campanha presidencial.","notícias da campanha presidencial, eleição presidencial.") = 4
f("notícias sobre campanha presidencial.","notícias sobre campanha de comidas orgânicas, campanha, campanha, campanha") = 6


## Por distância com ponderamento (TF-IDF)

### Importes

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(stop_words = stopWords)
tf_docs = vectorizer.fit_transform(C).toarray() # Bag of words dos documentos
tf_query = tf_docs[-1][:] # Bag of words da query
tf_docs = tf_docs[:-1][:] # Bag of words dos documentos

In [ ]:
np.dot(tf_docs[0],tf_query)

0.6521711197303464

In [ ]:
print('BoW de cada documento:\n',tf_docs)
print('\nBow da query:\n',tf_query)
print('\n')

BoW de cada documento:
 [[0.         0.         0.         0.59909697 0.         0.
  0.80067647]
 [0.34889082 0.55842384 0.         0.30229226 0.55842384 0.
  0.40400521]
 [0.52875615 0.         0.         0.45813442 0.         0.71451367
  0.        ]
 [0.27880467 0.         0.54419311 0.24156696 0.         0.7535033
  0.        ]
 [0.83018195 0.33219088 0.         0.17982529 0.33219088 0.
  0.24033151]]

Bow da query:
 [0.45094253 0.         0.         0.39071374 0.         0.60936332
 0.52217807]




In [ ]:
for doc, i in zip(C[:-1],range(tf_docs.shape[0])):
    f = np.dot(tf_docs[i][:],tf_query) # Produto interno por meio de matrix entre a query e o documento
    print('f("{}","{}") = {}'.format(q,doc,f))

f("notícias sobre campanha presidencial.","notícias sobre") = 0.6521711197303464
f("notícias sobre campanha presidencial.","notícias sobre campanha de comidas orgânicas") = 0.486402111825811
f("notícias sobre campanha presidencial.","notícias da campanha presidencial") = 0.8528364795638876
f("notícias sobre campanha presidencial.","notícias da campanha presidencial, eleição presidencial.") = 0.6792656928379586
f("notícias sobre campanha presidencial.","notícias sobre campanha de comidas orgânicas, campanha, campanha, campanha") = 0.5701204110655602


## Por BM25

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
query = "windy London"
tokenized_query = query.split(" ")

In [ ]:
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

[0.         0.93729472 0.        ]


In [ ]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

# ---------------------------------------


# Usando Dataset Projeto

## Bibliotecas

In [ ]:
!pip install rank_bm25
!pip install unidecode
!pip install --upgrade gensim

     |████████████████████████████████| 245kB 9.2MB/s 
     |████████████████████████████████| 24.2MB 8.5MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import json
import nltk
import string
from rank_bm25 import BM25Okapi
from unidecode import unidecode
import gensim
from gensim.summarization.bm25 import BM25
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Abertura do dataset

In [ ]:
with open('/content/drive/Shareddrives/COVID - Projeto/Repositório Perguntas_Respostas - Página1.csv','r') as f:
    df = pd.read_csv(f)
df.head()

,Classificação (FioCruz),Sentença,Resposta
0,O que é ?,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,O que é ?,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,Boatos,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Boatos,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,Boatos,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
new_df = df[df['Sentença'] != 'Sentença'].copy()
new_df.shape

(722, 3)

## Pre-processamento do dataset

In [ ]:
new_df.columns = ['classe','pergunta','resposta']
new_df.drop(['classe'],axis=1,inplace=True)
new_df.head()

,pergunta,resposta
0,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
new_df.isna().sum()

pergunta    24
resposta    24
dtype: int64

In [ ]:
new_df.dropna(inplace = True)
new_df.reset_index(inplace= True,drop=True)
new_df.head()

,pergunta,resposta
0,O que é o novo coronavírus?,Os coronavírus causam infecções respiratórias ...
1,Por que a doença causada pelo novo vírus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,A Organização Mundial de Saúde (OMS) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,Diante da falta de álcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"Chá de abacate com hortelã, mel, uísque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
perguntas = new_df['pergunta']
for i in range(perguntas.size):
    perguntas[i] = unidecode(perguntas[i]).lower()
new_df['pergunta'] = perguntas
new_df.head()

,pergunta,resposta
0,o que e o novo coronavirus?,Os coronavírus causam infecções respiratórias ...
1,por que a doenca causada pelo novo virus receb...,"Desde o início de fevereiro, a Organização Mun..."
2,a organizacao mundial de saude (oms) disse que...,Não. A Organização Mundial da Saúde não disse ...
3,diante da falta de alcool gel podemos colocar ...,Preparações caseiras não possuem a certificaçã...
4,"cha de abacate com hortela, mel, uisque ou out...","Até o momento, não há nenhum medicamento, subs..."


In [ ]:
print(("Dados duplicados: %d")%(new_df.duplicated(['pergunta']).sum()))
new_df.drop_duplicates(['pergunta'],inplace=True)
new_df.reset_index(inplace= True,drop = True)
print("Shape dataset: {}".format(new_df.shape))

Dados duplicados: 133
Shape dataset: (565, 2)


### Salvando dataset


In [ ]:
new_df.to_csv('/content/drive/Shareddrives/COVID - Projeto/repositorio_projeto.csv')

### Abrindo dataset

In [ ]:
new_df = pd.read_csv('/content/drive/Shareddrives/COVID - Projeto/repositorio_projeto.csv')

In [ ]:
perguntas = new_df['pergunta']

In [ ]:
document = []
for pergunta,i in zip(perguntas,range(perguntas.size)):
    try:
      tokens = nltk.word_tokenize(pergunta,language = 'portuguese')
      tokens = [w for w in tokens if not w in string.punctuation]
      document.append(tokens)
    except:
      print(pergunta, i)
document = np.array(document)

## Salvando amostra

In [ ]:
sample = np.random.choice(document,size=50,replace=False)

In [ ]:
with open('/content/drive/Shareddrives/COVID - Projeto/amostra.json','w') as outfile:
    json.dump(sample,outfile)

## Abrindo amostra


In [ ]:
with open('/content/drive/Shareddrives/COVID - Projeto/amostra.json','r') as f:
    sample = json.load(f)

## Utilizando BM25

In [ ]:
bm25 = BM25(document)
#bm25 = BM25Okapi(document)

In [ ]:
#@title Texto de título padrão { run: "auto" }
posicao =  1#@param {type:"integer"}
try:
    query = sample[posicao]
    print('Query:'," ".join(query) + '?','\n')
    score = bm25.get_scores(query)
    top_n = np.argsort(score)[::-1][2:6] # Apresentar as 3 melhores assimilações
    for i in top_n:
        print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25
except:
    print("Posicao invalida")

Query: o que sao coronavirus? 

Pergunta: o que sao testes rapidos (igm/igg)? | Score BM25: 4.49
Pergunta: o que sao os testes rapidos? | Score BM25: 4.49
Pergunta: o que sao testes rt- pcr? | Score BM25: 4.49
Pergunta: quais sao as complicacoes do novo coronavirus? | Score BM25: 4.42


### Analise de metricas

In [ ]:
df_avaliado = pd.read_csv('/content/drive/Shareddrives/COVID - Projeto/repositorio_avaliado_v2.csv')
df_avaliado.head()

,pergunta,assimilacao_um,nota_um,assimilacao_dois,nota_dois,assimilacao_tres,nota_tres
0,quais sao os sintomas do novo coronavirus?,quais os sintomas do coronavirus?,100,quais sao os sintomas?,100,quais sao as complicacoes do novo coronavirus?,75
1,o que sao coronavirus?,o que e o novo coronavirus?,100,o que sao os coronavirus?,100,o que e covid-19?,100
2,qual o plano da secretaria da saude para monit...,como a rede municipal de saude se preparou par...,60,qual a orientacao para os profissionais da sau...,0,o ministerio da saude tem uma plataforma para ...,80
3,quem precisa ficar em isolamento?,casos suspeitos tem sido mantidos em isolament...,60,o que significa isolamento domiciliar?,10,que cuidados devo ter se estiver em isolamento?,0
4,como e feito o diagnostico do covid-19?,como e feito o diagnostico da covid-19?,100,como e feito o diagnostico?,100,como e feito o tratamento contra o coronavirus?,0


In [ ]:
media_um = df_avaliado['nota_um'].mean()
desvio_um = df_avaliado['nota_um'].std()
media_dois = df_avaliado['nota_dois'].mean()
desvio_dois = df_avaliado['nota_dois'].std()
media_tres = df_avaliado['nota_tres'].mean()
desvio_tres = df_avaliado['nota_tres'].std()
print("Media 1: {} +- {:.2f}\nMedia 2: {} +- {:.2f}\nMedia 3: {} +- {:.2f}".format(media_um,desvio_um,media_dois,desvio_dois,media_tres,desvio_tres))
print("Media geral: {:.2f}".format(np.array([media_um,media_dois,media_tres]).mean()))

Media 1: 56.7 +- 40.46
Media 2: 40.26 +- 42.96
Media 3: 25.1 +- 37.78
Media geral: 40.69


In [ ]:
df_avaliado.drop(['pergunta','assimilacao_um','assimilacao_dois','assimilacao_tres'],axis = 1,inplace = True)

In [ ]:
descriminante = 0 # Qualquer valor maior que... 
matrix_avaliacao = np.zeros(df_avaliado.shape,dtype = int)
matrix_cumali = np.zeros(df_avaliado.shape,dtype=int) # Cumulative Gain@k (CG@k)
matrix_disc_cumali = np.zeros(df_avaliado.shape) # Discounted Cumulative Gain@k (DCG@k)
matrix_disc_i_cumali = np.zeros(df_avaliado.shape) # Ideal Discounted Cumulative Gain@k (IDCG@k)
df_array = df_avaliado.values
for posi in range(df_array.shape[0]):
    soma_cg = 0
    soma_dcg = 0
    soma_idcg = 0
    sorted_matrix = np.sort(df_array[posi])[::-1]
    for posi_interno in range(df_array[posi].size):

      #----------------------------------------------#
      if(df_array[posi][posi_interno] > descriminante):
          matrix_avaliacao[posi][posi_interno] = 1
      #----------------------------------------------#

      #----------------------------------------------#
      soma_cg += df_array[posi][posi_interno]
      matrix_cumali[posi][posi_interno] = soma_cg # (CG@k)

      #----------------------------------------------#
      soma_dcg += (df_array[posi][posi_interno]/np.log2(posi_interno + 2))
      matrix_disc_cumali[posi][posi_interno] = soma_dcg # (DCG@k)

      #----------------------------------------------#
      soma_idcg += (sorted_matrix[posi_interno]/np.log2(posi_interno + 2))
      matrix_disc_i_cumali[posi][posi_interno] = soma_idcg # (IDCG@k)
      
    

In [ ]:
matrix_pre = np.zeros(df_avaliado.shape) # precision@k
matrix_recall = np.zeros(df_avaliado.shape) # recall@k
for posi in range(matrix_avaliacao.shape[0]):
    soma = matrix_avaliacao[posi].sum()
    if(soma != 0):
        tp = 0
        fp = 0
        for i,j in zip(matrix_avaliacao[posi],range(matrix_avaliacao[posi].size)):
            if(i == 1):
                tp += 1
                matrix_pre[posi][j] = round((tp/(tp+fp)),2)
                matrix_recall[posi][j] = round((tp/soma),2)
            else:
                fp += 1
                matrix_pre[posi][j] = round((tp/(tp+fp)),2)
                matrix_recall[posi][j] = round((tp/soma),2)

In [ ]:
matrix_ap = np.zeros(df_avaliado.shape[0]) # average precision
for posi in range(matrix_avaliacao.shape[0]):
    soma = 0
    for i in range(matrix_avaliacao[posi].size):
        if(matrix_avaliacao[posi][i] == 1):
            soma += matrix_pre[posi][i]
    matrix_ap[posi] = round((soma/matrix_avaliacao[posi].size),2)


In [ ]:
matrix_ndgc = np.zeros(df_avaliado.shape) # Normalized Discounted Cumulative Gain
for posi in range(matrix_ndgc.shape[0]):
    for posi_interno in range(matrix_ndgc.shape[1]):
        if(matrix_disc_i_cumali[posi][posi_interno] != 0):
            matrix_ndgc[posi][posi_interno] = round((matrix_disc_cumali[posi][posi_interno]/matrix_disc_i_cumali[posi][posi_interno]),2)

In [ ]:
MAP = matrix_ap.mean() # mean average precision
MAP_std = matrix_ap.std() # mean average precision std
print("MAP: {:.2f} +- {:.2f}".format(MAP,MAP_std))

MAP: 0.56 +- 0.34


In [ ]:
df_ndgc = pd.DataFrame(matrix_ndgc,columns=['N@1','N@2','N@3'])
df_ndgc.head()

,N@1,N@2,N@3
0,1.00,1.00,1.00
1,1.00,1.00,1.00
2,0.75,0.51,0.85
3,1.00,1.00,1.00
4,1.00,1.00,1.00


In [ ]:
dici = {'MAP': [round(MAP,4)], 'N@1' : [round(df_ndgc['N@1'].mean(),4)], 'N@2' : [round(df_ndgc['N@2'].mean(),4)], 'N@3' : [round(df_ndgc['N@3'].mean(),4)]}
new_df_ndgc = pd.DataFrame(dici,index=['BM25'])
new_df_ndgc

,MAP,N@1,N@2,N@3
BM25,0.5596,0.7488,0.7742,0.8332


In [ ]:
new_df_ndgc.to_csv('/content/drive/Shareddrives/COVID - Projeto/metricas_bm25.csv')

In [ ]:
new_df_ndgc.to_latex('/content/drive/Shareddrives/COVID - Projeto/metricas_bm25.tex')

### Correcao valores nulos

In [ ]:
df_avaliado = df_avaliado[df_avaliado['nota_um'] == 0]
df_avaliado = df_avaliado[df_avaliado['nota_dois'] == 0]
df_avaliado = df_avaliado[df_avaliado['nota_tres'] == 0]
df_avaliado.head()

,pergunta,assimilacao_um,nota_um,assimilacao_dois,nota_dois,assimilacao_tres,nota_tres
11,o alcool em gel e obrigatorio?,diante da falta de alcool gel podemos colocar ...,0,o alcool 92% e eficiente? e o alcool de cereai...,0,usar o alcool em gel e mais eficiente do que l...,0
20,como faco para comunicar um desvio de qualidad...,quais sao os equipamentos de protecao individu...,0,como faco para saber se estou com o novo coron...,0,o que e um contato domiciliar de caso suspeito...,0
29,havera abordagem e multa para cidadaos em veic...,havera medida de restricao ou bloqueio a pesso...,0,o alcool em gel e obrigatorio?,0,qual o risco para as criancas? em caso de infe...,0
35,qual e a definicao de pessoasmonitoradas?,qual e a definicao de caso excluido?,0,qual e a definicao de caso descartado?,0,qual e a definicao de caso confirmado?,0
36,como e que as pessoas podem ajudar a acabar co...,"com as aulas suspensas, as criancas podem brin...",0,como as pessoas que precisam trabalhar podem s...,0,quantos e quais hospitais podem receber pacien...,0


In [ ]:
print("Tamanho do DF avaliado: {}".format(df_avaliado.shape[0]))

Tamanho do DF avaliado: 5


In [ ]:
df_avaliado.values[9]

array(['como e que as pessoas podem ajudar a acabar com o estigma relacionado com o covid-19?',
       'com as aulas suspensas, as criancas podem brincar na rua? e recomendavel?',
       0,
       'como as pessoas que precisam trabalhar podem se prevenir de covid-19 ao usarem o transporte publico?',
       0,
       'quantos e quais hospitais podem receber pacientes com a covid-19?',
       0], dtype=object)

In [ ]:
#@title Texto de título padrão { run: "auto" }
variable_name = "profissio" #@param {type:"string"}
for pergunta in perguntas:
    if(variable_name in pergunta):
        print(pergunta)

qual a orientacao para os profissionais da saude quando retornam a seus domicilios?
as profissionais de saude gravidas correm maior risco de resultados adversos se cuidarem de pacientes com covid-19?
como e a prevencao contra o coronavirus para os profissionais de saude?
por que e importante o municipio fazer a notificacao dentro de 24 horas? quais providencias o profissional deve adotar?
os testes rapidos sao de uso profissional?
o profissional de enfermagem pode utilizar mascara caseira em servicos de saude?
se um profissional da equipe testar positivo para covid-19, o que a equipe devera fazer?
quais os cuidados a serem adotados pelos profissionais de enfer-magem que realizam assistencia em atencao domiciliar? e se o paciente estiver com quadro suspeito ou confirmado?
posso atuar sem carteira de identidade profissional (cip)?
a minha carteira de identidade profissional esta vencida. o que devo fazer?
como sei que o profissional de saude esta tomando a precaucao correta contra o covi

In [ ]:
#query = np.array(question_test)[6]
query = question_test.iloc[5]
print(query,'\n')
query = nltk.word_tokenize(unidecode(query).lower(),language = 'portuguese')
query = [w for w in query if not w in string.punctuation]
score = bm25.get_scores(query)

top_n = np.argsort(score)[::-1][:5] # Apresentar as 5 melhores assimilações
for i in top_n:
    print("Pergunta: {} | Score BM25: {:.2f}".format(perguntas[i],score[i])) # Print da pergunta similiar e o score do BM25

quantas mascaras de tecido devo ter? 

Pergunta: como o usuario deve proceder para ter esse tipo de atendimento? | Score BM25: 7.47
Pergunta: quais as recomendacoes para quem precisar viajar neste momento? | Score BM25: 6.97
Pergunta: quantoslaboratorios publicos fazem o exame de deteccao em pernambuco? | Score BM25: 6.63
Pergunta: o covid-19 e  transmitido pelo ar? | Score BM25: 5.78
Pergunta: o que sao medidas de higiene e etiqueta respiratoria?  | Score BM25: 5.08
